In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [3]:
# Load the dataset
dataset = load_dataset("cifar10")

In [4]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.randint(0, 90, (1, )) < 10:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return (image, label)

In [5]:
train_dataset = CatsDataset()

In [6]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

In [12]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [13]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 5),
    nn.MaxPool2d((5, 5)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(400, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
for epoch in range(40):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.6248473540233199
Epoch 2 Loss: 0.5537416803988682
Epoch 3 Loss: 0.533106681647574
Epoch 4 Loss: 0.5093943081843625
Epoch 5 Loss: 0.5034890229914598
Epoch 6 Loss: 0.4956237499121648
Epoch 7 Loss: 0.4860181838843473
Epoch 8 Loss: 0.4732182363795627
Epoch 9 Loss: 0.46584705296595386
Epoch 10 Loss: 0.45862883443285707
Epoch 11 Loss: 0.4439179318346036
Epoch 12 Loss: 0.4413159436481014
Epoch 13 Loss: 0.42812169547293594
Epoch 14 Loss: 0.4227311937672317
Epoch 15 Loss: 0.41557447716688656
Epoch 16 Loss: 0.4041731517026379
Epoch 17 Loss: 0.39928441100819095
Epoch 18 Loss: 0.3885746628615507
Epoch 19 Loss: 0.3835947811603546
Epoch 20 Loss: 0.37652607679746714
Epoch 21 Loss: 0.3712842434075228
Epoch 22 Loss: 0.3649736603923664
Epoch 23 Loss: 0.358978720986919
Epoch 24 Loss: 0.35481153741763655
Epoch 25 Loss: 0.33986823222819407
Epoch 26 Loss: 0.3299565072272234
Epoch 27 Loss: 0.32477358581533855
Epoch 28 Loss: 0.3294470378547717
Epoch 29 Loss: 0.3107146199341792
Epoch 30 Loss: 0

In [17]:
test_dataset = CatsDataset(train=False)

In [18]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [19]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7670, device='mps:0')
